In [73]:
import json
import re
import pickle

import numpy as np
import torch

### hard voting

In [6]:
def voting(*args):
    count={}
    for i in args: # list
        try: count[str(sorted(i))] += 1
        except: count[str(sorted(i))]=1
    
    new = sorted(count.items(), key=lambda x: -x[1])
    if new[0][1] == len(args):
        return eval(new[0][0])
    elif new[0][1] == len(args)-1:
        return eval(new[0][0])
    else:
        return args[-1]

In [7]:
def postprocessing(data):
    new_dict = dict()
    for slot, values in data.items():
        lst = []
        for value in values:
            gen = re.sub('\s(?=[\=\(\)\&])|(?<=[\=\(\)\&])\s', "", value)
            gen = gen.replace(" : ", ":")
            gen = gen.replace(" , ", ", ")
            gen = re.sub('(?<=역)역*', "", gen)
            lst.append(gen)    
        new_dict[slot] = lst

    return new_dict

In [24]:
f1 = open('./ensemble/test_preds_mapped.csv', 'r')
f2 = open('./ensemble/prediction_ensemble_sorted_and_postprocessed_2.csv', 'r')
f3 = open('./ensemble/prediction_ensemble_sorted_and_postprocessed_3.csv', 'r')
# f4 = open('./ensemble/trade_predictions_fold_5.csv', 'r')
# f5 = open('./ensemble/trade_predictions_fold_4.csv', 'r')

res1 = json.load(f1)
res2 = json.load(f2)
res3 = json.load(f3)
# res4 = json.load(f4)
# res5 = json.load(f5)

In [25]:
res1 = postprocessing(res1) # 도균님 - 꿔바로우 처리 안되어있음.
res_col = [res1, res2, res3, res4, res5]
keys = res1.keys()
poll = {}
for key in keys:
    voted = voting(res1[key], res2[key], res3[key], res4[key], res5[key])
    poll[key] = voted
poll = postprocessing(poll)

with open('./prediction_ensemble_sorted_and_postprocessed_3.csv', 'w') as f:
    json.dump(poll, f, ensure_ascii=False, indent=2)

In [4]:
test = json.load(open('ensemble/trade_predictions_fold_4.csv'))

In [8]:
test = postprocessing(test)

In [16]:
keys = test.keys()
for key in keys:
    print(key)
    break

square-silence-9151:숙소_관광_11-0


In [17]:
test['square-silence-9151:숙소_관광_11-0']

['숙소-가격대-적당', '숙소-스파 유무-yes', '숙소-종류-호텔']

In [ ]:
count={}
for i in args: # list
    try: count[str(sorted(i))] += 1
    except: count[str(sorted(i))]=1

new = sorted(count.items(), key=lambda x: -x[1])
if new[0][1] == len(args):
    return eval(new[0][0])
elif new[0][1] == len(args)-1:
    return eval(new[0][0])
else:
    return args[-1]

### soft voting

In [116]:
k2_s_logits = pickle.load(open('/opt/ml/repo/taepd/som-dst_new/outputs/logits/k2_s_logits', 'rb'))
k2_g_logits = pickle.load(open('/opt/ml/repo/taepd/som-dst_new/outputs/logits/k2_g_logits', 'rb'))
k2_g_idx = pickle.load(open('/opt/ml/repo/taepd/som-dst_new/outputs/logits/k2_g_idx', 'rb'))

k3_s_logits = pickle.load(open('/opt/ml/repo/taepd/som-dst_new/outputs/logits/k3_s_logits', 'rb'))
k3_g_logits = pickle.load(open('/opt/ml/repo/taepd/som-dst_new/outputs/logits/k3_g_logits', 'rb'))
k3_g_idx = pickle.load(open('/opt/ml/repo/taepd/som-dst_new/outputs/logits/k3_g_idx', 'rb'))

k8_s_logits = pickle.load(open('/opt/ml/repo/taepd/som-dst_new/outputs/logits/k8_s_logits', 'rb'))
k8_g_logits = pickle.load(open('/opt/ml/repo/taepd/som-dst_new/outputs/logits/k8_g_logits', 'rb'))
k8_g_idx = pickle.load(open('/opt/ml/repo/taepd/som-dst_new/outputs/logits/k8_g_idx', 'rb'))

In [ ]:
#  _g_logit, _g_id = torch.topk(g, 100, dim=-1)
# g_logit = _g_logit.detach().cpu().numpy()
# g_id = _g_id.detach().cpu().numpy()

In [142]:
tmp = []
def trans_tensor(g_logits, g_idx):
    for logit, idx in zip(g_logits, g_idx):
        # dummy tensor 생성
        f = list(logit.size())[:3]
        f.append(35004)
        t = torch.zeros(f)

        try:
            for i, g in zip(idx[0,0], logit[0,0]):
                for j, k in zip(i, g):
                    t[:,:,:,j] = k
        except:
            pass
    return t

In [140]:
print(k2_g_idx[1])
print(k2_g_logits[2].size())

tensor([], device='cuda:0', size=(1, 0, 9, 100), dtype=torch.int64)
torch.Size([1, 4, 9, 100])


In [143]:
t1 = trans_tensor(k2_g_logits, k2_g_idx)
# t2 = trans_tensor(k3_g_logits, k3_g_idx)
# t3 = trans_tensor(k8_g_logits, k8_g_idx)


print(t1.size())

torch.Size([1, 0, 9, 35004])


In [112]:
tmp = []
for logit, idx in zip(k3_g_logits, k3_g_idx):
    # dummy tensor 생성
    f = list(logit.size())[:3]
    f.append(35004)
    t = torch.zeros(f)
    
    for i, g in zip(idx[0,0], logit[0,0]):
        for j, k in zip(i, g):
            t[:,:,:,j] = k
#       print(g)
        print(t[:,:,:,j].size())

#     print(id[0,0,0])
    # 100개 후보값
#     print(logit[0][0][0])

#     print(t.size())

#     z = e.detach().cpu().numpy()
    
#     print(z)
    break
#     tmp.append(z)

torch.Size([1, 3, 9])
torch.Size([1, 3, 9])
torch.Size([1, 3, 9])
torch.Size([1, 3, 9])
torch.Size([1, 3, 9])
torch.Size([1, 3, 9])
torch.Size([1, 3, 9])
torch.Size([1, 3, 9])
torch.Size([1, 3, 9])


In [63]:
zz = np.concatenate(tmp)
zz.shape

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 3 and the array at index 1 has size 0